In [19]:
import requests
import pandas as pd
from explogleif.entity import Entity

In [2]:
# exemple with Danone FR, LEI: 969500KMUQ2B6CBAF162
lei_danone = '969500KMUQ2B6CBAF162'


# Exploration de la base de données du LEI

## Count of LEIs + latest registration

In [112]:
url = 'https://api.gleif.org/api/v1/lei-records'

In [113]:
params = {
    # Country code
    # FR for France, DE for Germany and so on...
    'filter[entity.legalAddress.country]': None,
    
    # Entity category
    # BRANCH, FUND, SOLE_PROPRIETOR, GENERAL, RESIDENT_GOVERNMENT_ENTITY, INTERNATIONAL_ORGANIZATION
    'filter[entity.category]': None,
    
    # List of status
    # ISSUED, LAPSED, ANNULLED, PENDING_TRANSFER, PENDING_ARCHIVAL, DUPLICATE, RETIRED, MERGED
    'filter[registration.status]': None,
    
    
    # pagination
    
    # Which page to show, from the total result set. Must be at least 1.
    'page[number]': 1,
    
    # Number of LEI Records to show in each page of results (one page per API response body). Must be between 1 and 200.
    'page[size]': 1
}

In [117]:
response2 = requests.get(url,params=params)
response2

<Response [200]>

In [118]:
response2 = response2.json()

In [119]:
response2.keys()

dict_keys(['meta', 'links', 'data'])

In [120]:
response2['meta']['pagination']['total']

2188202

In [121]:
response2['data'][0]['attributes']['registration']

{'initialRegistrationDate': '2022-08-22T07:14:03Z',
 'lastUpdateDate': '2022-08-22T07:17:33Z',
 'status': 'ISSUED',
 'nextRenewalDate': '2023-08-22T07:17:23Z',
 'managingLou': '529900T8BM49AURSDO55',
 'corroborationLevel': 'FULLY_CORROBORATED',
 'validatedAt': {'id': 'RA000394', 'other': None},
 'validatedAs': 'U22110DL1999PTC099814',
 'otherValidationAuthorities': [{'validatedAt': {'id': 'RA000754'},
   'validatedAs': '07AACCP5921G2ZI'}]}

## Search for an entity (fuzzy completion)

In [128]:
url = "https://api.gleif.org/api/v1/fuzzycompletions"

In [129]:
user_input = "danone"

In [130]:
params = {
    'field': 'entity.legalName',
    'q': user_input
}

In [131]:
requests.get(url,params=payload)

<Response [200]>

In [132]:
response = requests.get(url,params=params)
response

<Response [200]>

In [135]:
my_json = response.json()
my_json

{'data': [{'type': 'fuzzycompletions', 'attributes': {'value': 'DANONE'}},
  {'type': 'fuzzycompletions',
   'attributes': {'value': 'DAMONET'},
   'relationships': {'lei-records': {'data': {'type': 'lei-records',
      'id': '894500P2FHRGMZXCWD75'},
     'links': {'related': 'https://api.gleif.org/api/v1/lei-records/894500P2FHRGMZXCWD75'}}}},
  {'type': 'fuzzycompletions',
   'attributes': {'value': 'Davonel'},
   'relationships': {'lei-records': {'data': {'type': 'lei-records',
      'id': '894500Z11BHYAU2GDT58'},
     'links': {'related': 'https://api.gleif.org/api/v1/lei-records/894500Z11BHYAU2GDT58'}}}},
  {'type': 'fuzzycompletions',
   'attributes': {'value': 'DANON LP'},
   'relationships': {'lei-records': {'data': {'type': 'lei-records',
      'id': '254900F6H310ZAE34N45'},
     'links': {'related': 'https://api.gleif.org/api/v1/lei-records/254900F6H310ZAE34N45'}}}},
  {'type': 'fuzzycompletions',
   'attributes': {'value': 'DACONE SL'},
   'relationships': {'lei-records': {'d

In [138]:
for entity in my_json['data']:
    print(entity['attributes'])

{'value': 'DANONE'}
{'value': 'DAMONET'}
{'value': 'Davonel'}
{'value': 'DANON LP'}
{'value': 'DACONE SL'}
{'value': 'DANONE AB'}
{'value': 'DANONE RE'}
{'value': 'DANONE SA'}
{'value': 'DANONI AS'}
{'value': 'DAYONE AS'}


## Search for an entity (filter[entity.names])

In [11]:
url = "https://api.gleif.org/api/v1/lei-records"

In [12]:
user_input = "danone"
page_number = 1
page_size = 200

In [13]:
params = {
        "filter[entity.names]": user_input,
        "page[number]": page_number,
        "page[size]": page_size,
    }

In [14]:
response = requests.get(url, params=params).json()

In [15]:
print("Showing only ",response["meta"]["pagination"]["perPage"]," out of ",response["meta"]["pagination"]["total"]," results.")

Showing only  200  out of  60  results.


In [16]:
entity_list = []

for json_entity in response["data"]:

    new_entity = Entity(
        name=json_entity["attributes"]["entity"]["legalName"]["name"],
        lei=json_entity["id"],
        city=json_entity["attributes"]["entity"]["legalAddress"]['city'],
        country=json_entity["attributes"]["entity"]["legalAddress"]['country'])

    entity_list.append(new_entity)

    print("Name : ", new_entity.name)
    print("LEI : ", new_entity.lei)
    print("Address : ", new_entity.city, ", ", new_entity.country)
    print("\n")

Name :  DANONE
LEI :  969500KMUQ2B6CBAF162
Address :  PARIS 9 ,  FR


Name :  DANONE DYNAMIQUE
LEI :  969500JN6C156IGKQ745
Address :  PARIS 13 ,  FR


Name :  DANONE MONETAIRE
LEI :  969500LJH58TDSRNXM82
Address :  PARIS 13 ,  FR


Name :  FONDS DANONE
LEI :  9695005QN0SFZZTEGT69
Address :  PARIS 9 ,  FR


Name :  DANONE
LEI :  213800TK6J92598JEM48
Address :  BRUXELLES ,  BE


Name :  GROUPE DANONE DYNAMIQUE
LEI :  969500XPJP95DRRQ9W94
Address :  PARIS 13 ,  FR


Name :  DANONE FINLAND OY
LEI :  2138004HG8MO8Q747L22
Address :  HELSINKI ,  FI


Name :  FPS DANONE COMMUNITIES
LEI :  969500MNFEVXGBRJKR54
Address :  PARIS 8 ,  FR


Name :  DANONE TRADING ELN B.V.
LEI :  213800E6RDD96TGHZ336
Address :  HOOFDDORP ,  NL


Name :  DANONE S.R.O.
LEI :  213800M6T9UKXB3L7G64
Address :  BRATISLAVA ,  SK


Name :  DANONE LIMITED
LEI :  213800KHX6QZIXORTF05
Address :  LONDON ,  GB


Name :  DANONE AB
LEI :  213800LUDZ3HKY1Q9W80
Address :  SOLNA ,  SE


Name :  DANONE S.P.A.
LEI :  2138009WALL12RQH4A

In [25]:
entity_dict = {"name": [], "lei": [], "city": [], "country": []}

for entity in entity_list:
    entity_dict["name"].append(entity.name)
    entity_dict["lei"].append(entity.lei)
    entity_dict["city"].append(entity.city)
    entity_dict["country"].append(entity.country)

df = pd.DataFrame.from_dict(entity_dict)
df

,name,lei,city,country
0,DANONE,969500KMUQ2B6CBAF162,PARIS 9,FR
1,DANONE DYNAMIQUE,969500JN6C156IGKQ745,PARIS 13,FR
2,DANONE MONETAIRE,969500LJH58TDSRNXM82,PARIS 13,FR
3,FONDS DANONE,9695005QN0SFZZTEGT69,PARIS 9,FR
4,DANONE,213800TK6J92598JEM48,BRUXELLES,BE
5,GROUPE DANONE DYNAMIQUE,969500XPJP95DRRQ9W94,PARIS 13,FR
6,DANONE FINLAND OY,2138004HG8MO8Q747L22,HELSINKI,FI
7,FPS DANONE COMMUNITIES,969500MNFEVXGBRJKR54,PARIS 8,FR
8,DANONE TRADING ELN B.V.,213800E6RDD96TGHZ336,HOOFDDORP,NL
9,DANONE S.R.O.,213800M6T9UKXB3L7G64,BRATISLAVA,SK
